In [1]:
from pdfminer import high_level
from PIL import Image
from dotenv import load_dotenv
import fitz
import io
from fpdf import FPDF

In [2]:
load_dotenv()

True

In [3]:
#extract holder details and split them in a list
path = r'C:\Users\Yatharth\Documents\VacCert-Project\cert.pdf'
text = high_level.extract_text(path)
ls = text.split("\n\n")

#dictionary for storing holder information
info = ["name", "age", "sex", "verif", "uhid", "refID", "vax", "dose1", "dose2"]
info_dict = {}

for i in range(len(info)):
    info_dict[info[i]] = ls[i]

In [7]:
#Open PDF again, get the QR code and save for use
pdf_file = fitz.open(path)
page = pdf_file[0]
image_list = page.getImageList()
xref = image_list[0][0]
base_image = pdf_file.extractImage(xref)
image_bytes = base_image["image"]
image_ext = base_image["ext"]
qr = Image.open(io.BytesIO(image_bytes))
qr.save(open(f"qr.{image_ext}", "wb"))

In [14]:
qr_link = r'C:\Users\Yatharth\Documents\VacCert-Project\vaccine-certificate-shortener\qr.png'

In [ ]:
class ph_1(FPDF):
    def 
    
ph_w = 281.25
ph_h = 500.25
pdf = FPDF(orientation="P",unit="pt",(ph_w,ph_h))
pdf.add_page()


In [16]:
#Creating new certificate with PyFPDF

new_pdf = FPDF('P','pt', (281.25,500.25))
new_pdf.add_page()
new_pdf.set_font('Arial', 'B', 11.2)
new_pdf.cell(w=225,h=166,txt="Final COVID-19 Vaccine Certificate",border=0,align='C',)
new_pdf.image(qr_link,x=49,y=292,w=184,h=184,type="png")
new_pdf.output('mob1.pdf', 'F')
#new_pdf.output('mob1.pdf', 'F')

''

In [26]:
import pyqrcode
from qrtools import QR

ImportError: cannot import name 'QR' from 'qrtools' (C:\Users\Yatharth\anaconda3\lib\site-packages\qrtools\__init__.py)

In [25]:
co = QR()
co.decode(qr)
print(co.data)

NameError: name 'QR' is not defined